In [1]:
#6: Use World Bank population data (bigquery-public-data.world_bank_global_population.population_by_country). 
# Query entire data set to your computer using BigQuery API. 
# Transform the data in Python to the format where the first column is year, other columns are populations in countries. 
# Create table for population in Nordics (headers will be ['year','Finland','Sweden','Norway','Denmark','Iceland’]). 
# Upload this table to BigQuery using API.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
 

In [3]:
import os
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

None


In [5]:
# Verify this prints a path
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

project_id = "atlantean-stock-385411"

sql = """
SELECT *
FROM `bigquery-public-data.world_bank_global_population.population_by_country`
"""

df_raw = pandas_gbq.read_gbq(sql, project_id=project_id)
print(df_raw.head())

None


KeyboardInterrupt: 

In [4]:
#This prettu much follow the example video
projectid = 'atlantean-stock-385411'
#GOOGLE_APPLICATION_CREDENTIALS

#SQL
sql = """
SELECT *
FROM `bigquery-public-data.world_bank_global_population.population_by_country`
"""

#Read the data
df_raw = pandas_gbq.read_gbq(sql, project_id = projectid)
df_raw

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=wKOJNQjWRcMvnQWpnDfIFymcWNChhC&prompt=consent&access_type=offline


gio: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=wKOJNQjWRcMvnQWpnDfIFymcWNChhC&prompt=consent&access_type=offline: Operation not supported


KeyboardInterrupt: 